In [1]:
%load_ext autoreload
%autoreload 2

# Notebook that handles data loading! 

This notebook is used to initially load and prepare data. This means loading into a datamanager, merging data (if wanted), and doing any preliminary alterations to the dataset. 

In [2]:
from data_prep import data_manager as DM

# instanciate a new datamanager 
dm = DM.Data_Manager()
# loads all data into the datamanager 
dm.data_loader()


## Removing features with null values

This mainly removes three features, these either have alot of null values or are not important for our final model, making it ok to remove either way.
This also removes the need for imputing. 

In [3]:
print("A DATA NULLS")
print (" ")
print('x_train_observed')
print(dm.X_train_observed_a.isnull().sum()[dm.X_train_observed_a.isnull().sum() > 0])
print (" ")
print('x_train_estimated')
print(dm.X_train_estimated_a.isnull().sum()[dm.X_train_estimated_a.isnull().sum() > 0])
print (" ")
print('x_test_estimated')
print(dm.X_test_estimated_a.isnull().sum()[dm.X_test_estimated_a.isnull().sum() > 0])
print (" ")
print('data_A')
print(dm.data_A.isnull().sum()[dm.data_A.isnull().sum() > 0])
print (" ")
print("B DATA NULLS")
print (" ")
print('x_train_observed')
print(dm.X_train_observed_b.isnull().sum()[dm.X_train_observed_b.isnull().sum() > 0])
print (" ")
print('x_train_estimated')
print(dm.X_train_estimated_b.isnull().sum()[dm.X_train_estimated_b.isnull().sum() > 0])
print (" ")
print('x_test_estimated')
print(dm.X_test_estimated_b.isnull().sum()[dm.X_test_estimated_b.isnull().sum() > 0])
print (" ")
print('data_B')
print(dm.data_B.isnull().sum()[dm.data_B.isnull().sum() > 0])
print (" ")
print("C DATA NULLS")
print (" ")
print('x_train_observed')
print(dm.X_train_observed_c.isnull().sum()[dm.X_train_observed_c.isnull().sum() > 0])
print (" ")
print('x_train_estimated')
print(dm.X_train_estimated_c.isnull().sum()[dm.X_train_estimated_c.isnull().sum() > 0])
print (" ")
print('x_test_estimated')
print(dm.X_test_estimated_c.isnull().sum()[dm.X_test_estimated_c.isnull().sum() > 0])
print (" ")
print('data_C')
print(dm.data_C.isnull().sum()[dm.data_C.isnull().sum() > 0])




A DATA NULLS
 
x_train_observed
ceiling_height_agl:m     22247
cloud_base_agl:m          8066
snow_density:kgm3       115945
dtype: int64
 
x_train_estimated
ceiling_height_agl:m     3919
cloud_base_agl:m         2094
snow_density:kgm3       15769
dtype: int64
 
x_test_estimated
ceiling_height_agl:m     793
cloud_base_agl:m         298
snow_density:kgm3       2880
dtype: int64
 
data_A
Series([], dtype: float64)
 
B DATA NULLS
 
x_train_observed
ceiling_height_agl:m     18772
cloud_base_agl:m          7473
snow_density:kgm3       111201
dtype: int64
 
x_train_estimated
ceiling_height_agl:m     3689
cloud_base_agl:m         1963
snow_density:kgm3       15713
dtype: int64
 
x_test_estimated
ceiling_height_agl:m     755
cloud_base_agl:m         277
snow_density:kgm3       2880
dtype: int64
 
data_B
Series([], dtype: float64)
 
C DATA NULLS
 
x_train_observed
ceiling_height_agl:m     19923
cloud_base_agl:m          8512
snow_density:kgm3       107593
dtype: int64
 
x_train_estimated
ceilin

In [4]:
dm.X_train_observed_a = dm.X_train_observed_a.drop("snow_density:kgm3", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("snow_density:kgm3", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("snow_density:kgm3", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("snow_density:kgm3", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("snow_density:kgm3", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("snow_density:kgm3", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("ceiling_height_agl:m", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("ceiling_height_agl:m", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("ceiling_height_agl:m", axis=1) 
dm.X_train_estimated_a = dm.X_train_estimated_a.drop("ceiling_height_agl:m", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("ceiling_height_agl:m", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("ceiling_height_agl:m", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("cloud_base_agl:m", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("cloud_base_agl:m", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("cloud_base_agl:m", axis=1) 
dm.X_train_estimated_a = dm.X_train_estimated_a.drop("cloud_base_agl:m", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("cloud_base_agl:m", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("cloud_base_agl:m", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("cloud_base_agl:m", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("cloud_base_agl:m", axis=1) 
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("cloud_base_agl:m", axis=1) 

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("ceiling_height_agl:m", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("ceiling_height_agl:m", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("ceiling_height_agl:m", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("snow_density:kgm3", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("snow_density:kgm3", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("snow_density:kgm3", axis=1)

## Now we look at sample rates

the training weather data is sampled every 15 minutes, whilst the pv_measurements are every hour. 
We can either 
1. sample down. Making weather data be every hour using the mean value of every hour 
2. sample up. Making the pv_measurement every 15 min instead of every hour. 

In [5]:
# Sample down

resamples = dm.resample_data([dm.X_train_observed_a, 
                                   dm.X_train_observed_b, 
                                   dm.X_train_observed_c, 
                                   dm.X_train_estimated_a,
                                   dm.X_train_estimated_b,
                                   dm.X_train_estimated_c,
                                   dm.X_test_estimated_a,
                                   dm.X_test_estimated_b,
                                   dm.X_test_estimated_c], "H")

dm.X_train_observed_a = resamples[0]
dm.X_train_observed_b = resamples[1] 
dm.X_train_observed_c = resamples[2] 
dm.X_train_estimated_a = resamples[3]
dm.X_train_estimated_b = resamples[4]
dm.X_train_estimated_c = resamples[5]
dm.X_test_estimated_a = resamples[6]
dm.X_test_estimated_b = resamples[7]
dm.X_test_estimated_c = resamples[8]



# Sample up

# resamples = dm.resample_data([
#                             dm.X_test_estimated_a,
#                             dm.X_test_estimated_b,
#                             dm.X_test_estimated_c], "H")


# dm.X_test_estimated_a = resamples[0]
# dm.X_test_estimated_b = resamples[1]
# dm.X_test_estimated_c = resamples[2]

# resamples = dm.resample_data([dm.train_a, dm.train_b, dm.train_c], "15T")

# train_a_15min = resamples[0]
# train_b_15min = resamples[1]
# train_c_15min = resamples[2]

# dm.train_a = resamples[0]
# dm.train_b = resamples[1]
# dm.train_c = resamples[2]


# dm.train_a




## Combining data into one training set 

In [6]:
dm.data_A, dm.data_B, dm.data_C = dm.combine_data()
dm.sorting_columns_inMainSets()


print(dm.data_A.shape, dm.data_B.shape, dm.data_C.shape)





(34061, 44) (32819, 44) (26071, 44)


## Removing const y data

In [7]:
dm.remove_constant_periods(30) # HR FREQUENCY
#dm.remove_constant_periods(200) # 15MIN FREQUENCY

y_train_a anomalies: 4
y_train_b anomalies: 103
y_train_c anomalies: 66


In [8]:
%store dm

Stored 'dm' (Data_Manager)
